# 4.1 使用Function_call

In [8]:

from langchain.agents import create_tool_calling_agent, AgentExecutor
import os
import dotenv
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate


# 加载环境变量
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = "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"

# 1. 准备工具
search = TavilySearchResults(max_results=3)
search_tool = Tool(
    func=search.run,
    name="search",
    description="用于检索互联网上的信息"
)

python_repl = PythonREPL()
cacl_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算"
)

tools = [search_tool, cacl_tool]

# 2. 准备模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 准备 Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的多功能助手。"),
    ("system","chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
# 提供记忆
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
)
# 4. 创建 Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# 5. 执行
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    memory=memory,
)

# 6. 调用
# 注意：Tavily 搜索国外数据比较强，如果搜索国内特定数据（如某公司股价）可能不如 Google/Bing 准确
result = agent_executor.invoke({"input": "特斯拉当前股价是多少？比去年上涨了百分之多少"})
print(result["output"])

特斯拉（Tesla, TSLA）当前股价为 **440.55 美元**。根据过去一年的数据，特斯拉的股价在过去一年内上涨了 **12.73%**，去年同期的最低股价为 **214.25 美元**，而最高股价为 **498.83 美元**。

如果你需要更详细的信息或其他问题，请告诉我！


## 4.2 使用ReAct模式

In [15]:
from langchain.agents import create_react_agent
from langchain import hub
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities.python import PythonREPL
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

# 加载环境变量
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
os.environ["TAVILY_API_KEY"] = "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"

# 1. 准备工具
search = TavilySearchResults(max_results=3)
search_tool = Tool(
    func=search.run,
    name="search",
    description="用于检索互联网上的信息"
)

python_repl = PythonREPL()
cacl_tool = Tool(
    name="Calculator",
    func=python_repl.run,
    description="用于执行数学计算"
)

tools = [search_tool, cacl_tool]

# 2. 准备模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 使用langchain hub的官方ReAct提示词模板

prompt = hub.pull("hwchase17/react-chat")
# 创建记忆实例
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
)
# 4. 创建 Agent
agent = create_react_agent(llm, tools, prompt)

# 5. 执行
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,
)

# 6. 调用
# 注意：Tavily 搜索国外数据比较强，如果搜索国内特定数据（如某公司股价）可能不如 Google/Bing 准确
result = agent_executor.invoke({"input": "特斯拉当前股价是多少？比去年上涨了百分之多少"})
print(result["output"])
result = agent_executor.invoke({"input": "比亚迪当前股价是多少？比去年上涨了百分之多少"})
print(result["output"])

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: search
Action Input: "特斯拉当前股价 2023年10月"
```[{'url': 'https://www.youtube.com/watch?v=cI0VCG3em-s', 'content': "Hello everyone, in this video, we continue to analyze and predict the trend of Tesla's stock price on October 10, 2023. First, let's review the basic information about the trend of Tesla's stock price on October 9. Its opening price is 255 and closing price is 31. The price is 259 67. The highest price in the session is 261 36. The lowest price is 252 05. The amplitude between the highest value and the lowest value in the session reaches 9 points. We will see that its closing price is still It closed above each moving average. The trading volume on that day was 101.2 billion shares, which was 0.85 times the volume of the previous trading day. The KD value double line continued to rise and the opening became smaller. This information shows that the stock price continues to rise. The MACD [